In [5]:
from pathlib import Path
import czifile
import tifffile
import napari
import os
import numpy as np
import pandas as pd
from utils import check_filenames, segment_nuclei_2d, segment_marker_positive_nuclei

In [2]:
# Copy the path where your images are stored, ideally inside the raw_data directory
directory_path = Path("./raw_data/Reelin")
roi_directory_path = Path("./raw_data/Reelin/ROI")

# Define the subdirectories containing your data
subdirectories = ["Contra", "Ipsi", "Sham"]

# Create empty lists to store all image filepaths and ROIs within the dataset directory
images = []
rois = []

# Create an empty list to store all stats extracted from each image
stats = []

# Scan subdirectories and add paths to images fitting certain conditions
for subdir in subdirectories:
    # Construct the subdirectory path
    image_path = directory_path / subdir
    # Iterate through the .czi files in the subdirectories
    for file_path in image_path.glob("*.czi"):
        # Remove unwanted images
        if "AWT" not in str(file_path) and "BWT" not in str(file_path):
            images.append(str(file_path))

# Scan ROI directory and add paths to the list
for file_path in roi_directory_path.glob("*.tif"):
    # Remove unwanted images
        if "AWT" not in str(file_path) and "BWT" not in str(file_path):
            rois.append(str(file_path))

# Check if there is any missing ROI or image file in their corresponding directories
check_filenames(images, rois) 

No files missing in images list.
No files missing in rois list.


In [ ]:
for image_path, roi_path in zip(images, rois):
    #TODO: Include the full loop here
    print (image_path)
    print(roi_path)         

In [14]:
image_path = images[0]
roi_path = rois[0]

# Read image and ROI files into Numpy arrays
img = czifile.imread(image_path)
roi = tifffile.imread(roi_path)

# Remove singleton dimensions and perform MIP on input image
img = img.squeeze()
img_mip = np.max(img, axis=1)

# Perform MIP for the region of interest
roi_mip = np.max(roi, axis=0)

# We will create a mask where label_mip is greater than or equal to 1
mask = roi_mip >= 1

# Apply the mask to img_mip
masked_img = np.where(mask, img_mip, 0)



In [15]:
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_mip)
viewer.add_image(roi_mip)
viewer.add_image(masked_img)

<Image layer 'roi_mip' at 0x1c88d6a55e0>

In [17]:
# Mask input image with the user defined ROI



<Image layer 'masked_img' at 0x1c8bac1e220>

In [ ]:
# Extract each of the channels separately
neun_mip = img_mip[0, :, :, :]
reelin_mip = img_mip[1, :, :, :]
gad67_mip = img_mip[2, :, :, :]
nuclei_mip = img_mip[3, :, :, :] 